2. Sistema de clasificación de subreddit. 

Se deberá implementar una función ```classify_subreddit(text)``` que clasifique un texto de entrada en una de las siguientes 
categorías: **MachineLearning, datascience, statistics, learnmachinelearning, computerscience, AskStatistics, artificial, analytics, datasets, deeplearning, rstats, computervision, DataScienceJobs, MLQuestions, dataengineering, data, dataanalysis, 
datascienceproject, Kaggle**.

Para ello, se proporciona un dataset sobre el que se podrán entrenar distintos algoritmos
de clasificación. La etiqueta del subreddit correspondiente se encuentra en la columna "subreddit". Se deberán probar, al menos, los siguientes 3 métodos:
- Un método basado en TF-IDF + algoritmo de clasificación de machine learning
- Un método basado en entidades reconocidas (Named-Entity Recognition) + algoritmo declasificación de machine learning 
- Un método basado en Word Embeddings + algoritmo de clasificación de machine learning

Para evaluar cada método, se utilizará la métrica f1 score, y se utilizará un 70% de los datos
del dataset para entrenamiento y un 30% para test (realizando un sampling aleatorio
previo)."""

from IPython.display import Markdown
display(Markdown(ejercicio))

Los posts de entrenamiento se encuentran en la columna 'clean_post' del dataframe, y las labels en la columna 'subreddit'. Las siguientes son las categorías en las que clasificaremos cada post:

In [ ]:
categories = ['MachineLearning', 'datascience', 'statistics', 'learnmachinelearning', 'computerscience', 'AskStatistics', 'artificial', 'analytics', 'datasets', 'deeplearning', 'rstats', 'computervision', 'DataScienceJobs', 'MLQuestions', 'dataengineering', 'data', 'dataanalysis', 'datascienceproject', 'Kaggle']

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split

reddit_df = pd.read_csv('processed_dataset.csv', encoding='UTF-8', sep=';', low_memory=False)
reddit_df['clean_post'] = reddit_df['clean_post'].fillna('')
reddit_df['subreddit'] = reddit_df['subreddit'].fillna('')
x_data, y_data = reddit_df['clean_post'], reddit_df['sentiment']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(
    x_data, 
    y_data,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

### Método basado en TF-IDF + algoritmo de clasificación de machine learning

<h4>Elección del algoritmo de machine learning</h4>

Para los métodos de clasificación usaremos regresión logística implementada en la librería sklearn por los siguientes motivos: 

- Ideal para clasificación binaria/multiclase

- Interpretable: Los coeficientes indican importancia de features

- Eficiente con datasets grandes

- Bueno con datos dispersos (sparse) como text data

- Rápido de entrenar y predecir


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Features y labels
X = reddit_df['clean_post']
y = reddit_df['subreddit']

# Dividimos con train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

vectorizer = TfidfVectorizer()

# Transformamos los datos
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Clasificador
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {f1}')

F1 Score: 0.5072098375581701


<h3>Análisis del F1 score bajo (0.507)</h3>

Posibles causas:

- Desbalanceo en las clases

- Configuración básica de TfidfVectorizer

- Parámetros por defecto en Logistic

In [33]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline

def train_tfidf_model(df: pd.DataFrame, 
                      text_column: str = 'clean_post',
                      label_column: str = 'subreddit',
                      random_state: int = 42) -> tuple:
    """
    Entrena un modelo TF-IDF optimizado para clasificación de texto.
    
    Args:
        df: DataFrame con los datos
        text_column: Nombre de la columna con el texto
        label_column: Nombre de la columna con las etiquetas
        random_state: Semilla aleatoria
        
    Returns:
        tuple: (modelo entrenado, f1_score)
    """
    # 1. Preparación de datos
    # Eliminar textos vacíos y muy cortos
    df = df[df[text_column].str.len() > 10].copy()
    
    # Preparar X e y
    X = df[text_column]
    y = df[label_column]
    
    # 2. Split de datos
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.3,
        random_state=random_state,
        stratify=y
    )
    
    # 3. Pipeline con hiperparámetros optimizados
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=10000,
            min_df=2,
            max_df=0.95,
            ngram_range=(1, 2),
            sublinear_tf=True
        )),
        ('clf', LogisticRegression(
            max_iter=1000,
            n_jobs=-1,
            random_state=random_state
        ))
    ])
    
    # 4. Entrenamiento
    pipeline.fit(X_train, y_train)
    
    # 5. Evaluación
    y_pred = pipeline.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    return pipeline, f1
model,f1 = train_tfidf_model(reddit_df)
print('F1 Score: {f1}')

F1 Score: {f1}


### Un método basado en entidades reconocidas (Named-Entity Recognition) + algoritmo declasificación de machine learning 

In [ ]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from typing import List, Tuple


### Un método basado en Word Embeddings + algoritmo de clasificación de machine learning